The low_contast_penetration JavaScript code works by assessing whether the mean grey value of a row of 20 pixels is less than a specified threshold. This threshold is determined as a proportion of the grey values of the pixels near 0mm depth. In order to obtain this proportion, I analysed images which a Clinical Scientist had classified and attempted to identify what the mean threshold was. Firstly, I recorded the following data obtained from these images in a spreadsheet. 

In [130]:
import pandas as pd

df = pd.read_csv('baseline_measurements.csv')
df.head()

,image_disc,date,series,image_no_marked,image_no_unmarked,manufacturer,probe_type,called_depth_mm,mean_contrast_20px_at zero,mean_contrast_20px_at_called_depth,proportion
0,1,27/08/2015,S0000000,US000005,US000004,TOSHIBA,CURVED LINEAR,126.6,49.4,39.1,0.791498
1,1,27/08/2015,S0000000,US000007,US000006,TOSHIBA,CURVED LINEAR,132.8,43.9,35.1,0.799544
2,1,27/08/2015,S0000000,US000009,US000008,TOSHIBA,CURVED LINEAR,134.9,38.4,32.8,0.854167
3,1,27/08/2015,S0000000,US000028,US000027,TOSHIBA,LINEAR,47.2,42.7,31.4,0.735363
4,1,27/08/2015,S0000000,US000029,US000030,TOSHIBA,LINEAR,46.9,42.7,35.7,0.836066


Obtain summary statistics of proportion as crude measure.

In [131]:
proportion_mean = df['proportion'].mean()
proportion_std = df['proportion'].std()

xs = list(df.index)
ys = list(df.proportion)
y_errs = [(i * (1 - i)) for i in ys]

err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, y_errs):
    err_xs.append((x, x))
    err_ys.append((y - yerr/2, y + yerr/2))

In [132]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, show
from bokeh.models import Range1d
output_notebook()

fig = figure(title='Threshold determined by Clinical Scientist',
            x_axis_label='Image Number',
            y_axis_label='Proportion of "zero" grey value')
fig.circle(xs,ys,radius=0.1)
#fig.multi_line(err_xs, err_ys,line_width=3)
fig.set(y_range=Range1d(0, 1))
show(fig)

BokehJS successfully loaded.

In [133]:
alg_df = pd.read_csv('algorithm1.csv')
alg_df

,image_no,disc,probe_type,actual,algorithm1,algorithm2,algorithm3,algorithm4,algorithm5,algorithm6
0,US000004,1,CURVED_LINEAR,126.6,130.3,130.3,129.0,133.2,129.0,130.7
1,US000006,1,CURVED_LINEAR,132.8,129.4,128.2,125.7,129.8,125.7,130.3
2,US000008,1,CURVED_LINEAR,134.9,139.0,139.0,136.5,140.7,136.5,139.4
3,US000027,1,LINEAR,47.2,44.2,43.6,0.0,0.0,0.0,47.1
4,US000030,1,LINEAR,46.9,44.1,42.2,0.0,0.0,0.0,44.8
5,US000031,1,LINEAR,47.2,45.0,44.5,0.0,0.0,0.0,47.7
6,US000033,1,LINEAR,46.6,44.4,43.9,0.0,0.0,0.0,47.1
7,US000004,3,CURVED_LINEAR,84.1,0.0,0.0,0.0,0.0,0.0,75.4
8,US000016,3,CURVED_LINEAR,131.3,128.4,127.6,126.9,130.6,126.9,130.6


The immediate impression is that the algorithm is generally underestimating the depth and, in some cases, not making a judgment at all. Each algorithm is essentially the same but with a number of adjustable parameters. These parametes are:
Threshold value
Number of data points below threshold value before call can be made
Number of data points to back-track once call has been made
Minimum number of pixels from 'zero' point before call can be made

Obtaining summary statistics of the classification error of each algorithm.

In [134]:
from scipy.stats import ttest_rel
from math import sqrt
alg_df_summary = pd.DataFrame(columns=['p_val','mean','std','se']);
for i in range(1,7):
    alg_name = 'algorithm' + str(i)
    diff = alg_df['actual'] - alg_df[alg_name]
    alg_df_summary.loc[i] = [ttest_rel(alg_df['actual'],alg_df[alg_name])[1],diff.mean(),diff.std(),(diff.std()/sqrt(6))]
alg_df_summary

,p_val,mean,std,se
1,0.298520,10.311111,27.822942,11.358669
2,0.269930,10.922222,27.645378,11.286178
3,0.015010,31.055556,30.202902,12.330283
4,0.025944,29.255556,32.177947,13.136592
5,0.015010,31.055556,30.202902,12.330283
6,0.711084,0.500000,3.907685,1.595306


In [135]:
xs = list(alg_df_summary.index)
ys = list(alg_df_summary['mean'])
y_errs = list(alg_df_summary['se'])

err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, y_errs):
    err_xs.append((x, x))
    err_ys.append((y - (yerr)/2, y + (yerr)/2))

fig2 = figure(title = 'Algorithm Performance', y_axis_label = 'Mean difference from actaul +/- SE', x_axis_label = 'Algorithm')
fig2.circle(xs,ys,radius=0.05)
fig2.multi_line(err_xs, err_ys, line_width=5)
fig2.set(y_range=Range1d(-10,40))
show(fig2)

Algorithm must have num_meeting_threshold around 10, curved linear threshold around > 0.8

Alterations to the algorithm seemed to fail to classify images obtained using linear probes more often than curved linear probes. 

Algorithm5 features:
linear cutoff = 0.85, curved cutoff = 0.8
val must be greater than 20 pixels from zero
there must have been at least 10 obs at lower threshold and then alg tracks back 10 pixels

Algorithm6 features:
linear cutoff = 0.85, curved cutoff = 0.8
val must be greater than 20 pixels from zero
there must have been at least 10 obs at lower threshold and then alg tracks back 10 pixels